In [11]:
import tensorflow as tf
import gc,uuid
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
import time_series_transform
from time_series_transform.time_series_transformer import Time_Series_Transformer
from time_series_transform.sequence_transfomer import *
import numpy as np
import IPython.display as display

In [12]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [15]:
CATEGORICAL_DIM = ['item_id','dept_id','cat_id','store_id','state_id']
WINDOW_SIZE = 10
TEST_RANGE = WINDOW_SIZE + 50
df = pd.read_csv('./Data/Train/sales_train_validation.csv')
df.index = df.id
df = df.drop('id',axis = 1)
train = df.drop(CATEGORICAL_DIM,axis =1).head(2)

In [16]:
fe = Time_Series_Transformer(train,[],seqTransformerList = [])
train_univariate,train_step = fe.get_tf_dataset(WINDOW_SIZE)